In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import pytz
import astropy
import sys
import astropy.units as u
from skyfield.api import Topos, load, EarthSatellite
from datetime import datetime, timedelta
from astropy.coordinates import get_body, get_sun, EarthLocation, SkyCoord, AltAz
sys.path.append('/Users/physarah/Development/forrest_test')
import lumos.calculator
import lumos.conversions
import lumos.constants


import json
import os
sys.path.append('/Users/physarah/Development/satellite-optical-brightness/analysis')

import satellite_models.diffuse_sphere as diffuse_sphere
import satellite_models.starlink_v1p5 as starlink_v1p5

def append_record(record, file):
    with open(file, 'a') as f:
        json.dump(record, f)
        f.write(os.linesep)

In [2]:
observer_location = Topos('33.7703 S', '151.1112 E', elevation_m=70)
macquarie_observatory = EarthLocation(lat=-33.7703*u.deg, lon=151.1112*u.deg, height=70*u.m)

sat_cat = "/Users/physarah/Development/satellite-optical-brightness/data/brightness_config_list.csv"
open_list = pd.read_csv(sat_cat)
ts = load.timescale()

In [3]:
cleaned_list = open_list[6:-1] # Do this because there is text at the top of the file

In [4]:
list_of_sats = cleaned_list['# This file contains a list of NORAD CAT IDs'].to_list()

In [5]:
pos_url = 'https://celestrak.org/NORAD/elements/gp.php?GROUP=starlink&FORMAT=tle'
sats_all = load.tle(pos_url)

In [6]:
def generate_datetime_range(start_time, end_time, interval):
    """
    Generate a list of datetimes between start_time and end_time at a given interval.
    """
    current_time = start_time
    datetime_list = []

    while current_time <= end_time:
        datetime_list.append(current_time)# - timedelta(hours = time_delta))
        current_time += interval

    return datetime_list

start_time = datetime(2024, 6, 10, 0, 0, 1)  
end_time = datetime(2024, 6, 11, 0, 0, 1)   
#time_interval = timedelta(minutes=1)          
time_interval = timedelta(minutes=1)          

list_of_times = generate_datetime_range(start_time, end_time, time_interval)
times = []
for timestamp_pd in list_of_times:
    TIME_PD = timestamp_pd.replace(tzinfo=pytz.utc)
    time_sf = ts.utc(TIME_PD)
    times.append(time_sf)

In [7]:
sat_alts_list = []
sat_azs_list = []
sat_names = []
sat_height_list = []

for norad_catid in list_of_sats:
    try:
        sats_catid = sats_all[int(norad_catid)]
        alt_test = []
        az_test = []
        sat_height = []
        sat_names_indi = []
        for time_now in times:
            topocentric = (sats_catid - observer_location).at(time_now)
            position = sats_catid.at(time_now)
            distance_to_satellite_m = position.distance().m - lumos.constants.EARTH_RADIUS
            alt, az, _ = topocentric.altaz()  
            alt_test.append(alt.degrees)
            az_test.append(az.degrees)
            sat_height.append(distance_to_satellite_m)
            sat_names_indi.append(norad_catid)
            
        sat_names.append(sat_names_indi)
        sat_alts_list.append(alt_test)
        sat_azs_list.append(az_test)
        sat_height_list.append(sat_height)
    except:
        pass    

In [8]:
def count_negative_to_positive_transitions(alts):
    transition_count = 0

    for i in range(1, len(alts)):
        if alts[i-1] < 0 and alts[i] > 0:
            transition_count += 1

    return transition_count

result = count_negative_to_positive_transitions(sat_alts_list[0])
print(f"Negative to positive transitions: {result}")

Negative to positive transitions: 6


In [9]:
def flatten_list(nested_list):
    flattened_list = [item for sublist in nested_list for item in sublist]
    return(flattened_list)

In [10]:
input_sat_ID = flatten_list(sat_names)
input_sat_altitude = flatten_list(sat_alts_list)
input_sat_azimuth = flatten_list(sat_azs_list)
input_sat_height = flatten_list(sat_height_list)

In [11]:
observation_times = [dt.utc_iso().split('.') for dt in times]

sun_alt_list, sun_az_list = ([],[])
for i, (time) in \
    enumerate(zip(observation_times)):
    observation_time = astropy.time.Time(time, format = 'isot')
    sun_alt, sun_az = lumos.calculator.get_sun_alt_az(observation_time, macquarie_observatory)
    sun_alt_list.append(sun_alt)
    sun_az_list.append(sun_az)

In [12]:
input_sun_altitude = [float(item[0]) for item in sun_alt_list] * len(set(input_sat_ID))
input_sun_azimuth = [float(item[0]) for item in sun_az_list] * len(set(input_sat_ID))
input_times = times * len(set(input_sat_ID))

In [13]:
terminator_angle = []
for j in np.arange(0,len(input_times)):
    
    i_input_sat_ID = input_sat_ID[j]
    i_input_sat_height = input_sat_height[j]
    i_input_times = input_times[j]
    i_input_sat_altitude = input_sat_altitude[j]
    i_input_sat_azimuth = input_sat_azimuth[j]
    i_input_sun_altitude = input_sun_altitude[j]
    i_input_sun_azimuth = input_sun_azimuth[j]
    (s1, s2, s3, term_theta) = lumos.calculator.get_brightness_coords(i_input_sat_altitude, 
                                           i_input_sat_azimuth, 
                                           i_input_sat_height, 
                                           i_input_sun_altitude, 
                  
                                                                      i_input_sun_azimuth)
    terminator_angle.append(term_theta)

In [14]:
for j in np.arange(0,len(input_times)):
    
    i_input_sat_ID = input_sat_ID[j]
    i_input_sat_height = input_sat_height[j]
    i_input_times = input_times[j]
    i_input_sat_altitude = input_sat_altitude[j]
    i_input_sat_azimuth = input_sat_azimuth[j]
    i_input_sun_altitude = input_sun_altitude[j]
    i_input_sun_azimuth = input_sun_azimuth[j]
      
    i_input_times_format = i_input_times.utc_iso()
        
    intensities_lab_brdfs = lumos.calculator.get_intensity_observer_frame(
            starlink_v1p5.SURFACES_LAB_BRDFS,
            i_input_sat_height, 
            i_input_sat_altitude, 
            i_input_sat_azimuth,
            i_input_sun_altitude,
            i_input_sun_azimuth,
            include_earthshine = True,
            earth_brdf=lumos.brdf.library.PHONG(0.53, 0.28, 7.31)) # use the vegitation example that Forrest uses 
    
    magnitudes_lab_brdfs = lumos.conversions.intensity_to_ab_mag(intensities_lab_brdfs)

    big_dict = {'sun_altitude':i_input_sun_altitude,
                'sun_azimuth':i_input_sun_azimuth,
                'sat_height':i_input_sat_height,
                'magnitude':magnitudes_lab_brdfs,
                'sat_altitude':i_input_sat_altitude,
                'sat_azimuth':i_input_sat_azimuth,
                'times':i_input_times_format,
                'cat_name':i_input_sat_ID}
    

    append_record(big_dict, '/Users/physarah/Desktop/LAB_BRDF_winter_time22222.json')